# download xml file

In [31]:
#pip install lxml
#history 8是那个特别小的

In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import os
import math
import csv 
from bs4 import BeautifulSoup
import bz2
import lxml
import requests

In [51]:
import os
import requests

URL = "https://dumps.wikimedia.org/enwiki/20210101/enwiki-20210101-pages-meta-history4.xml-p557617p558391.bz2"
filename = os.path.basename(URL)

response = requests.get(URL, stream=True)

if response.status_code == 200:
    with open(filename, 'wb') as out:
        out.write(response.content)
else:
    print('Request failed: %d' % response.status_code)

In [55]:
import bz2

In [56]:
zipfile = bz2.BZ2File(filename) # open the file
data = zipfile.read() # get the decompressed data
newfilepath = filename[:-4] # assuming the filepath ends with .bz2
open(newfilepath, 'wb').write(data) # write a uncompressed file


1992086992

In [57]:

infile = open(newfilepath,"rb")
contents = infile.read()
soup = BeautifulSoup(contents,'html.parser')

In [58]:
all_revision=soup.find_all('revision')

In [59]:
revision_list=[]
for i in all_revision:
    each_revision={}
    #find contributor of every revision
    contributor=i.find('contributor')
    try:
        username=contributor.find('username').get_text()
    except:
        username=np.nan
    each_revision['Contributor_Name']=username
    
    #find each revision time
    time=i.find('timestamp').get_text()
    each_revision['time']=time
    
    #find each revision comment
    try:
        comment=i.find('comment').get_text()
    except:
        comment=np.nan
    each_revision['comment']=comment 
    
    revision_list.append(each_revision)

In [60]:
df=pd.DataFrame(revision_list)

In [61]:
def find_count(name,name_dict):
    name_dict[name]+=1
    return name_dict[name]-1

In [62]:
name_count={}
all_contributor=df['Contributor_Name'].unique()
for i in all_contributor:
    name_count[i]=0
    
count_copy=name_count.copy()
df['Revision Time']=df['Contributor_Name'].apply(find_count,args=(count_copy,))

In [63]:
df

,Contributor_Name,time,comment,Revision Time
0,Vodex,2004-03-27T17:54:55Z,NaN,0
1,Vodex,2004-04-05T18:48:35Z,expanded article - need more examples. How to ...,1
2,Vodex,2004-04-08T09:53:14Z,ref. Kirt Angle,2
3,Dale Arnett,2004-04-13T06:02:24Z,=Examples of angles=,0
4,Dale Arnett,2004-05-07T19:09:32Z,=Examples of angles=,1
...,...,...,...,...
65105,Pengo,2006-01-02T00:22:32Z,pPpPp,3
65106,Pengo,2006-01-06T01:41:10Z,NaN,4
65107,Pengo,2006-01-06T01:45:50Z,NaN,5
65108,Pengo,2006-01-06T01:48:42Z,NaN,6


In [66]:
#df.to_csv('raw_df.csv',index=False)

In [203]:
import pandas as pd
from datetime import datetime
comments = pd.read_csv("raw_df.csv")


In [204]:
otherinfo = comments.rename(columns={"time": "date"})
otherinfo = otherinfo.rename(columns={"Contributor_Name": "commentor"})

In [205]:
otherinfo['date'] = otherinfo['date'].apply(lambda x: x.replace('^^^_',''))
otherinfo['date'] = otherinfo['date'].apply(lambda x: x.replace('T',' '))
otherinfo['date'] = otherinfo['date'].apply(lambda x: x.replace('Z',''))

otherinfo

,commentor,date,comment,Revision Time
0,Vodex,2004-03-27 17:54:55,NaN,0
1,Vodex,2004-04-05 18:48:35,expanded article - need more examples. How to ...,1
2,Vodex,2004-04-08 09:53:14,ref. Kirt Angle,2
3,Dale Arnett,2004-04-13 06:02:24,=Examples of angles=,0
4,Dale Arnett,2004-05-07 19:09:32,=Examples of angles=,1
...,...,...,...,...
65105,Pengo,2006-01-02 00:22:32,pPpPp,3
65106,Pengo,2006-01-06 01:41:10,NaN,4
65107,Pengo,2006-01-06 01:45:50,NaN,5
65108,Pengo,2006-01-06 01:48:42,NaN,6


# English light dump

In [5]:
import requests

In [80]:
# download the "en" light dump data
# save as txt file 
import urllib.request
import zipfile

url = 'http://wwm.phy.bme.hu/LD/ld_en_wiki.zip' 
filehandle, _ = urllib.request.urlretrieve(url)
zip_file_object = zipfile.ZipFile(filehandle, 'r')
first_file = zip_file_object.namelist()[0]
file = zip_file_object.open(first_file)
content = file.read()

In [81]:
tempzip = open("enwiki.txt", "wb")
tempzip.write(content)

9316424979

In [199]:
def create_title_col(mal):
    mal.reset_index(drop=True,inplace=True)
    titles_df= mal[mal['revert'].isna()]
    titles=titles_df['date'].values
    
    start=titles_df.index[0]
    raw=np.array(titles_df.index)
    raw=np.append(raw,mal.index[-1])
    counts=np.diff(raw)-1
    counts[len(counts)-1]=counts[len(counts)-1]+1
    newmal=mal.iloc[start:,:]
    newmal=newmal.drop(index=titles_df.index)

    title_col=[]
    for i in range(len(titles)):
        each=[titles[i]]*counts[i]
        title_col.extend(each)

    newmal['title']=title_col
    return newmal

In [200]:
chunk_list=pd.read_csv('enwiki.txt', sep=" ", header=None,chunksize=500000, error_bad_lines=False)
all_en=[]
a=0
for i in chunk_list:
    
    if a==100:
        break

    i.columns=['date','revert','edit','commentor']
    try:
        i['date'] = i['date'].apply(lambda x: str(x).replace('^^^_',''))
        i['date'] = i['date'].apply(lambda x: str(x).replace('T',' '))
        i['date'] = i['date'].apply(lambda x: str(x).replace('Z',''))

        newmal=create_title_col(i)
        all_en.append(newmal)
        
        a=a+1
    except:
        break

In [207]:
all_en[0]

,date,revert,edit,commentor,title
1,2010-02-02 08:11:10,0.0,2.0,Altenmann,Blackboard
2,2010-02-02 08:10:50,0.0,1.0,Altenmann,Blackboard
4,2010-02-02 08:09:58,0.0,2.0,Skäpperöd,Coimbatore_tile_company
5,2010-02-02 08:08:41,0.0,1.0,Razals,Coimbatore_tile_company
7,2010-02-02 08:09:24,0.0,2.0,Geecboy,he_moonstone_2
...,...,...,...,...,...
499995,2009-12-31 22:15:42,0.0,39.0,WikiWriteyWeb,Quake_Wars:_Ray_ raced
499996,2009-12-31 22:15:14,0.0,38.0,WikiWriteyWeb,Quake_Wars:_Ray_ raced
499997,2009-12-31 22:13:28,0.0,37.0,WikiWriteyWeb,Quake_Wars:_Ray_ raced
499998,2009-12-31 22:12:58,0.0,36.0,WikiWriteyWeb,Quake_Wars:_Ray_ raced


In [209]:
result=pd.DataFrame()
for i in all_en:
    each=pd.merge(i,otherinfo,left_on=['date','commentor'],right_on=['date','commentor'],how='inner')
    result=pd.concat([result,each])
result.reset_index(drop=True,inplace=True)

In [210]:
result

,date,revert,edit,commentor,title,comment,Revision Time
0,2009-11-01 20:45:23,0.0,1.0,Prezbo,Operation_Green,moved [[Operation Green]] to [[Operation Green...,1
1,2009-12-27 18:23:24,0.0,5.0,SmackBot,Peasenhall_Murder,/* Parody usage */Delink dates ([[WP:MOSUNLINK...,243
2,2009-03-23 20:11:10,0.0,1.0,Kicking222,Give_out_But_Don't_Give_Up,moved [[Give out But Don't Give Up]] to [[Give...,1
3,2009-12-11 03:10:52,0.0,20.0,SmackBot,Savage_Model_1907,Correct cap in header and/or general fixes. us...,151
4,2009-02-05 03:47:49,0.0,1.0,Gilliam,Mantell_UFO_Incident,moved [[Mantell UFO Incident]] to [[Mantell UF...,35
5,2008-12-29 23:45:27,0.0,2.0,Lightbot,Kannambra-I,Units/dates/other,69
6,2009-12-23 19:53:02,0.0,69.0,SmackBot,Princess_DisneyMania,Delink dates ([[WP:MOSUNLINKDATES]]) using [[P...,119
7,2009-12-23 20:43:25,0.0,8.0,SmackBot,Rare_&_Unreleased_Recordings_from_the_Golden_R...,Delink dates ([[WP:MOSUNLINKDATES]]) using [[P...,270
8,2008-09-18 01:00:38,0.0,4.0,Lightbot,Adolphe-Simon_Neboux,Units/dates/other,52
9,2009-12-23 17:18:51,0.0,9.0,SmackBot,Panayiotis_Xiourouppas,Date maintenance tags and general fixes,29


# pageview data

In [ ]:
#-- raw discription Number of view on English Wikipedia of article Paris from November 6th to November 20th 2015

In [71]:
#pip install git+https://github.com/Commonists/pageview-api.git


In [222]:
#pip install attrdict
from attrdict import AttrDict 
import pageviewapi

In [223]:
word_range = result['title'].values  #example --> will change later

In [226]:
dataframe = pd.DataFrame()
for i in word_range:
    try:
        page_v_dict = pageviewapi.per_article('en.wikipedia', i, '20200101', '20210101',
                            access='all-access', agent='all-agents', granularity='monthly')
        new_dictionary = {}
        for key in page_v_dict: 
            for i in page_v_dict[key]:
                new_dictionary['article'] = i['article']
                new_dictionary['timestamp'] = i['timestamp']
                try:
                    new_dictionary['views']+=i['views']
                except:
                    new_dictionary['views']=0


            new_dataf =  pd.DataFrame(new_dictionary,index=[0])
            dataframe = pd.concat([new_dataf,dataframe])
    except:
        continue

In [232]:
page_view = dataframe.drop_duplicates()
page_view.reset_index(drop=True,inplace=True)
page_view.head()

,article,timestamp,views
0,Give_Out_but_Don't_Give_Up,2020120100,36
1,"Berno,_Apostle_of_the_Obotrites",2020120100,710
2,Eko_Eko_Azarak_(manga),2020120100,10780
3,Arturo_Maly,2020120100,6531
4,Rubbery_figures,2020120100,173


# clickstream

# sentiment analysis

In [8]:
#using vader
#pip install vaderSentiment
#https://github.com/cjhutto/vaderSentiment

In [11]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [16]:
#vader sentiment analysis ex
sentences = ["Code is great!", 
             "Damn, this is so hard", 
             "This bood is good",
             "Make sure you :) or :D today!",
             "today SUX!",
             "Catch utf-8 emoji such as such as 💘 and 💋 and 😁"
             ]

analyzer = SentimentIntensityAnalyzer()
for sentence in sentences:
    vs = analyzer.polarity_scores(sentence)
    print(vs['compound'])

0.6588
-0.5524
0.4404
0.8633
-0.5461
0.875


We decide to use compound for the computation. Because from the vader's discription "The compound score is computed by summing the valence scores of each word in the lexicon, adjusted according to the rules, and then normalized to be between -1 (most extreme negative) and +1 (most extreme positive). "

we will use vader for the following cells to find the sentiment score of the comments part

In [215]:
new_dictionary = {}
for i in np.arange(otherinfo.shape[0]):
    try:
        comments = otherinfo.iloc[i]['comment']

        analyzer = SentimentIntensityAnalyzer()
        vader_score = analyzer.polarity_scores(comments)
        
        new_dictionary[comments] = vader_score['compound']
    except:
        new_dictionary[comments] = 0

In [216]:
null_dataframe = pd.DataFrame()
null_dataframe['comment'] = new_dictionary.keys()
null_dataframe['sentiment_score'] = new_dictionary.values()

In [217]:
#merge this score dataframe with original dataframe
score_dataf = pd.merge(otherinfo,null_dataframe,on = 'comment', how = 'left')
score_dataf

,commentor,date,comment,Revision Time,sentiment_score
0,Vodex,2004-03-27 17:54:55,NaN,0,0.00
1,Vodex,2004-04-05 18:48:35,expanded article - need more examples. How to ...,1,-0.34
2,Vodex,2004-04-08 09:53:14,ref. Kirt Angle,2,0.00
3,Dale Arnett,2004-04-13 06:02:24,=Examples of angles=,0,0.00
4,Dale Arnett,2004-05-07 19:09:32,=Examples of angles=,1,0.00
...,...,...,...,...,...
65105,Pengo,2006-01-02 00:22:32,pPpPp,3,0.00
65106,Pengo,2006-01-06 01:41:10,NaN,4,0.00
65107,Pengo,2006-01-06 01:45:50,NaN,5,0.00
65108,Pengo,2006-01-06 01:48:42,NaN,6,0.00
